# Parsing data from decision tree spreadsheets

In [1]:
import xlrd
import os
import pandas as pd

In [2]:
_PATH = '../data/decision_tree/'
DATA_FILE_NAMES = sorted(os.listdir(_PATH))
print(DATA_FILE_NAMES)

['diagnostic_data_list_mar2021.xlsx', 'diagnostic_data_matrix_mar2015.xlsx']


## Parsing `diagnostic_data_matrix_mar2015.xlsx`

In [3]:
FILE_NAME = 'diagnostic_data_matrix_mar2015.xlsx'
book = xlrd.open_workbook(_PATH + FILE_NAME)

In [4]:
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))

The number of worksheets is 5
Worksheet name(s): ['All plants', 'Vegetables', 'Fruit and nut trees', 'Woody ornamentals', 'Flowers']


In [5]:
INDEX = 1
URL_INDEX = 'http://www.ipm.ucdavis.edu'


def retrieve_problems(col_index, sh):
  name = sh.cell_value(rowx = 0, colx = col_index)
  link = sh.cell_value(rowx = 1, colx = col_index)
  macrotype = sh.cell_value(rowx = 2, colx = col_index)
  microtype = sh.cell_value(rowx = 3, colx = col_index)
  d = {1: 'vegetable', 2: 'fruit', 3: 'woody', 4: 'flower'}
  plant = d[sh.number]
  
  problem = {
      'name': name,
      'link': URL_INDEX + link,
      'macrotype': macrotype,
      'microtype': microtype,
      'plant': plant,
      
  }
  return problem

def check(to_parse):
  assert isinstance(to_parse, str) and len(to_parse) > 0
  tmp = to_parse.lower().split(':')
  assert len(tmp) == 2
  return tmp

def retrieve_part(to_parse):
  tmp = to_parse.strip()
  assert len(tmp) > 0
  return [tmp.strip()]

def retrieve_descs(to_parse):
  descs = [i.strip() for i in to_parse.split(",")]
  assert len(descs) > 0 and "" not in descs
  return descs

def parse_sheet(index = 1, debug = False):
  sh = book.sheet_by_index(index)
  print(f"{sh.name} {sh.nrows} {sh.ncols}")
  print(f"Cell A1 is {sh.cell_value(rowx=0, colx=0)}")

  START_COL = 4
  START_ROW = 8
  END_COL = sh.ncols
  END_ROW = sh.nrows
  
  df = {
      'problem_name': [],
      'problem_link': [],
      'problem_macrotype': [],
      'problem_microtype': [],
      'problem_plant': [],
      'problem_part': [],
      'problem_description': [],
      'problem_desc_link': []
  }

  for c in range(START_COL, END_COL):
    problem = retrieve_problems(c, sh)
    for r in range(START_ROW, END_ROW):
      tmp = sh.cell_value(rowx = r, colx = c)
      if tmp == '':
        continue
      desc_link = tmp
      tmp = sh.cell_value(rowx = r, colx = 0)
      parts, descs = check(tmp)
      parts = retrieve_part(parts)
      descs = retrieve_descs(descs)
      tmp = sh.cell_value(rowx = r, colx = 1)
      if tmp != '':
        tmp_part, tmp_descs = check(tmp)
        tmp_part = retrieve_part(tmp_part)
        tmp_descs = retrieve_descs(tmp_descs)
        parts.extend(tmp_part)
        descs.extend(tmp_descs)
      tmp = sh.cell_value(rowx = r, colx = 2)
      if tmp != '':
        tmp_part, tmp_descs = check(tmp)
        tmp_part = retrieve_part(tmp_part)
        tmp_descs = retrieve_descs(tmp_descs)
        parts.extend(tmp_part)
        descs.extend(tmp_descs)
      
      parts = list(set(parts))
      if debug: print(r, parts)
      assert len(parts) == 1
      descs = list(set(descs))

      df['problem_name'].append(problem['name'])
      df['problem_link'].append(problem['link'])
      df['problem_macrotype'].append(problem['macrotype'])
      df['problem_microtype'].append(problem['microtype'])
      df['problem_plant'].append(problem['plant'])
      
      df['problem_desc_link'].append(desc_link)
      df['problem_part'].append(parts[0])
      df['problem_description'].append(descs)
    
  return df

In [6]:
df = parse_sheet(index = 1)
df = pd.DataFrame(data = df)
df.sample(10)

Vegetables 122 191
Cell A1 is Name of pest or problem 


,problem_name,problem_link,problem_macrotype,problem_microtype,problem_plant,problem_part,problem_description,problem_desc_link
147,Potato tuberworm,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/PE...,Invertebrate,Lepidoptera,vegetable,stems,"[holes, chewing, or boring, boring, feeding, c...",http://www.ipm.ucdavis.edu/PMG/P/I-LP-POPE-CD....
208,Seed piece decay,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Bacteria,vegetable,tubers,"[spots, lesions]",http://www.ipm.ucdavis.edu/PMG/E/D-PO-ECAR-TU....
66,Onion maggot,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/PE...,Invertebrate,Diptera,vegetable,bulbs,"[chewed, feeding damage]",http://www.ipm.ucdavis.edu/PMG/D/I-DP-DANT-LV....
299,Early blight,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Fungus,vegetable,tubers,"[spots, lesions]",http://www.ipm.ucdavis.edu/PMG/A/D-PO-ASOL-TU....
126,Corn earworm / tomato fruitworm,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/PE...,Invertebrate,Lepidoptera,vegetable,pods,"[holes, chewed]",http://www.ipm.ucdavis.edu/PMG/H/I-LP-HZEA-CD....
326,Fusarium wilt,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Fungus,vegetable,stems,"[bleached, discolored, discoloration, streaks,...",http://www.ipm.ucdavis.edu/PMG/F/D-SP-FSOL-RO....
144,Omnivorous leafroller,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/PE...,Invertebrate,Lepidoptera,vegetable,fruit,"[spotted, discolored, russetted, streaks, stip...",http://www.ipm.ucdavis.edu/PMG/P/I-LP-PSTU-CD....
38,Pepper weevil,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/PE...,Invertebrate,Coleoptera,vegetable,fruit,"[punctures, holes, webbing, gouges, punctured,...",http://www.ipm.ucdavis.edu/PMG/A/I-CO-AEUG-PU....
604,Frost,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/EN...,Abiotic,,vegetable,leaves,"[deformed, distorted, blisters, pustules, roug...",http://www.ipm.ucdavis.edu/PMG/W/A-PO-WEAT-FO....
517,Potato leafroll,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Virus,vegetable,leaves,"[dieback of tips and margins, discolored, necr...",http://www.ipm.ucdavis.edu/PMG/P/D-PO-PLRV-FO....


In [7]:
INDEX = 2
tmp = parse_sheet(index = INDEX)
tmp = pd.DataFrame(data = tmp)

print(df.shape, tmp.shape)
df = df.append(tmp)
print(df.shape)

Fruit and nut trees 133 232
Cell A1 is Name of pest or problem 
(652, 8) (952, 8)
(1604, 8)


In [8]:
INDEX = 3
tmp = parse_sheet(index = INDEX)
tmp = pd.DataFrame(data = tmp)

print(df.shape, tmp.shape)
df = df.append(tmp)
print(df.shape)

Woody ornamentals 132 186
Cell A1 is Name of pest or problem 
(1604, 8) (759, 8)
(2363, 8)


In [9]:
INDEX = 4
tmp = parse_sheet(index = INDEX)
tmp = pd.DataFrame(data = tmp)

print(df.shape, tmp.shape)
df = df.append(tmp)
print(df.shape)

Flowers 73 56
Cell A1 is Name of pest or problem 
(2363, 8) (227, 8)
(2590, 8)


In [10]:
df.sample(10)

,problem_name,problem_link,problem_macrotype,problem_microtype,problem_plant,problem_part,problem_description,problem_desc_link
14,Cyclamen mite (COMBINED),http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/PE...,Invertebrate,Acari,fruit,overall,"[slow growth, stunted]",http://www.ipm.ucdavis.edu/PMG/S/I-AC-SPAL-CD....
278,Downy mildew,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Fungus,vegetable,leaves,"[dieback of tips and margins, discolored, necr...",http://www.ipm.ucdavis.edu/PMG/P/D-CU-PCUB-FO....
425,Seed piece decay (DUPLICATE)-other in bacteria...,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Fungus,vegetable,tubers,"[spots, lesions]",http://www.ipm.ucdavis.edu/PMG/F/D-PO-FSPP-TU....
914,Rind stipple of grapefruit,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/EN...,Abiotic,,fruit,fruit,"[spotted, discolored, russetted, streaks, stip...",http://www.ipm.ucdavis.edu/PMG/W/A-CI-WEAT-FR....
60,Eucalyptus tortoise beetle,http://www.ipm.ucdavis.edu/PMG/PESTNOTES/pn741...,Invertebrate,Coleoptera,woody,leaves,"[irregular, scalloped, holes or raggety appear...",http://www.ipm.ucdavis.edu/PMG/T/I-CO-TSLO-CD....
74,Pacific flatheaded borer,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/PE...,Invertebrate,Coleoptera,fruit,twigs/branches,"[cankers, oozing, streaks, sap, lesions, crack...",http://www.ipm.ucdavis.edu/PMG/C/I-CO-CMAL-LV....
674,Mineral deficiencies,http://www.ipm.ucdavis.edu/PMG/GARDEN/ENVIRON/...,Abiotic,,woody,leaves,"[flecks, dots, mottled, stippled, spots, lesio...",http://www.ipm.ucdavis.edu/PMG/M/A-WO-MDEF-FO....
352,Leaf spots,http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/DI...,Disease,Fungus,vegetable,stems,"[bleached, discolored, discoloration, streaks,...",http://www.ipm.ucdavis.edu/PMG/A/D-CA-ADAU-FO....
700,Air pollution,http://www.ipm.ucdavis.edu/PMG/GARDEN/ENVIRON/...,Abiotic,,woody,needles,[drop],http://www.ipm.ucdavis.edu/PMG/A/A-WO-APOL-FO....
626,Sudden oak death,http://www.ipm.ucdavis.edu/PMG/PESTNOTES/pn741...,Disease,Fungus,woody,twigs/branches,"[cankers, oozing, streaks, sap, lesions, crack...",http://www.ipm.ucdavis.edu/PMG/P/D-WO-PRAM-TR....


In [11]:
print(f'Problem macrotype:\n{df["problem_macrotype"].unique()}\n')
print(f'Problem microtype:\n{df["problem_microtype"].unique()}\n')
print(f'Problem plant:\n{df["problem_plant"].unique()}\n')
print(f'Problem part:\n{df["problem_part"].unique()}\n')
print(f'Shape:\n{df.shape}\n')
print(f'Shape of duplicated parts:\n{df[df["problem_part"].isin(["twigs/branches", "flowers/blossoms", "roots/crown", "nuts/kernels"])].shape}')


Problem macrotype:
['Invertebrate' 'Invertebrates' 'Disease' 'Diseases' 'Abiotic'
 'Invertebrate ']

Problem microtype:
['Acari' 'Coleoptera' 'Isopoda' 'Dermaptera' 'Diptera' 'Hemiptera'
 'Homoptera' 'Lepidoptera' 'Orthoptera' 'Stylommatophora' 'Symphyla'
 'Thysanoptera' 'Tylinchida' 'Bacteria' 'Fungus' 'Virus' '' 'Hymenoptera'
 'Mycoplasma' 'Viroid' 'Aphelenchida' 'Unknown' 'Phytoplasma' 'Other']

Problem plant:
['vegetable' 'fruit' 'woody' 'flower']

Problem part:
['overall' 'bulbs' 'leaves' 'fruit' 'spears' 'ferns' 'pods'
 'flowers/blossoms' 'shoots' 'seedlings' 'roots/crown' 'tubers' 'buds'
 'stems' 'kernels' 'roots' 'twigs/branches' 'canes' 'trunk' 'crown'
 'nuts/kernels' 'needles' 'nuts']

Shape:
(2590, 8)

Shape of duplicated parts:
(471, 8)


In [12]:
rename = {
    'Invertebrates' : 'Invertebrate', 
    'Diseases'      : 'Disease'     ,
    'Invertebrate ' : 'Invertebrate'
}
df['problem_macrotype'].replace(rename, inplace = True)

rename = {'Unknown': 'Other'}
df['problem_microtype'].replace(rename, inplace = True)

df['problem_part'] = df['problem_part'].str.split('/')
df = df.explode('problem_part')

df['problem_description'] = df['problem_description'].str.join(', ').replace({'"nests"': 'nests'})

print(f'Problem macrotype:\n{df["problem_macrotype"].unique()}\n')
print(f'Problem microtype:\n{df["problem_microtype"].unique()}\n')
print(f'Problem plant:\n{df["problem_plant"].unique()}\n')
print(f'Problem part:\n{df["problem_part"].unique()}\n')
print(f'Shape:\n{df.shape}\n')

Problem macrotype:
['Invertebrate' 'Disease' 'Abiotic']

Problem microtype:
['Acari' 'Coleoptera' 'Isopoda' 'Dermaptera' 'Diptera' 'Hemiptera'
 'Homoptera' 'Lepidoptera' 'Orthoptera' 'Stylommatophora' 'Symphyla'
 'Thysanoptera' 'Tylinchida' 'Bacteria' 'Fungus' 'Virus' '' 'Hymenoptera'
 'Mycoplasma' 'Viroid' 'Aphelenchida' 'Other' 'Phytoplasma']

Problem plant:
['vegetable' 'fruit' 'woody' 'flower']

Problem part:
['overall' 'bulbs' 'leaves' 'fruit' 'spears' 'ferns' 'pods' 'flowers'
 'blossoms' 'shoots' 'seedlings' 'roots' 'crown' 'tubers' 'buds' 'stems'
 'kernels' 'twigs' 'branches' 'canes' 'trunk' 'nuts' 'needles']

Shape:
(3061, 8)



In [13]:
df.sample(10)

,problem_name,problem_link,problem_macrotype,problem_microtype,problem_plant,problem_part,problem_description,problem_desc_link
657,Phytophthora root and crown rot (Red Stele),http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/DI...,Disease,Fungus,fruit,overall,"slow growth, stunted",http://www.ipm.ucdavis.edu/PMG/P/D-ST-PSPP-CR....
150,Gill's mealybug,http://www.ipm.ucdavis.eduanrcatalog.ucdavis.e...,Invertebrate,Homoptera,fruit,nuts,"moldy, spores, shriveled",http://www.ipm.ucdavis.edu/PMG/F/I-HO-FGIL-CD....
79,Lygus bugs (COMBINED),http://www.ipm.ucdavis.edu/PMG/GARDEN/VEGES/PE...,Invertebrate,Hemiptera,vegetable,fruit,"spotted, discolored, russetted, streaks, stipp...",http://www.ipm.ucdavis.edu/PMG/L/I-HM-LHES-CD....
590,Downy mildew,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/DI...,Disease,Fungus,fruit,shoots,"distorted, stunted, curled, galls, bushy",http://www.ipm.ucdavis.edu/PMG/P/D-GR-PVIT-FR....
533,Brown rot,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/DI...,Disease,Fungus,fruit,blossoms,"drop, dieback, shriveled, don't develop",http://www.ipm.ucdavis.edu/PMG/M/D-SF-MFRU-BL....
71,Brown marmorated stink bug,http://www.ipm.ucdavis.edu/PDF/pestalert/brown...,Invertebrate,Hemiptera,vegetable,fruit,"lack of fruit, bitter, pithy, may be lacking, ...",http://www.ipm.ucdavis.edu/PMG/H/I-HM-HHAL-AD....
560,Citrus blast,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/DI...,Disease,Fungus,fruit,branches,"girdled, dimpled areas, distorted, rough, swel...",http://www.ipm.ucdavis.edu/PMG/P/D-CI-PSYR-FO....
738,Verticillium wilt,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/DI...,Disease,Fungus,fruit,branches,"decay, drop, break, dieback, weak",http://www.ipm.ucdavis.edu/PMG/V/D-SF-VDAH-BT....
6,Citrus bud mite,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/PE...,Invertebrate,Acari,fruit,fruit,"deformed, creased, fused, split, leathery, cra...",http://www.ipm.ucdavis.edu/PMG/E/I-AC-ESHE-CD....
27,Pear rust mite,http://www.ipm.ucdavis.edu/PMG/GARDEN/FRUIT/PE...,Invertebrate,Acari,fruit,fruit,"spotted, discolored, russetted, streaks, stipp...",http://www.ipm.ucdavis.edu/PMG/E/I-AC-EPYR-CD....


## Parsing `diagnostic_data_list_mar2021.xlsx`

In [14]:
FILE_NAME = 'diagnostic_data_list_mar2021.xlsx'
book = xlrd.open_workbook(_PATH + FILE_NAME)

In [15]:
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))

The number of worksheets is 1
Worksheet name(s): ['Sheet1']


In [16]:
sh = book.sheet_by_index(0)
print(f"{sh.name} {sh.nrows} {sh.ncols}")
print(f"Cell A1 is {sh.cell_value(rowx=0, colx=0)}")

Sheet1 24134 5
Cell A1 is Choose your plant type. 


In [17]:
START_ROW = 1
END_ROW = sh.nrows

plant_type = []
plant_name = []
plant_part = []
plant_damage = []
link = []
for r in range(START_ROW, END_ROW):
  tmp_plant_type = sh.cell_value(rowx = r, colx = 0)
  tmp_plant_name = sh.cell_value(rowx = r, colx = 1)
  tmp_plant_part = sh.cell_value(rowx = r, colx = 2)
  tmp_plant_damage = sh.cell_value(rowx = r, colx = 3)
  tmp_link = sh.cell_value(rowx = r, colx = 4)
  plant_type.append(tmp_plant_type)
  plant_name.append(tmp_plant_name)
  plant_part.append(tmp_plant_part)
  plant_damage.append(tmp_plant_damage)
  link.append(tmp_link)

df = pd.DataFrame(data = {'plant_type': plant_type, 
                          'plant_name': plant_name, 
                          'plant_part': plant_part, 
                          'plant_damage': plant_damage,
                          'link': link
                          })

df.sample(10)

,plant_type,plant_name,plant_part,plant_damage,link
20619,Trees and shrubs (including roses),viburnum,leaves,"drop, cling to branches",http://ipm.ucanr.edu/PMG/PESTNOTES/pn7408.html
12161,Trees and shrubs (including roses),false cypress,buds,wilted,http://ipm.ucanr.edu/PMG/GARDEN/ENVIRON/frostd...
17557,Trees and shrubs (including roses),poplar,leaves,"discolored, necrosis, chlorosis",http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/INVERT/...
16284,Trees and shrubs (including roses),oak,twigs/branches,"cankers, wounds",http://ipm.ucanr.edu/PMG/GARDEN/ENVIRON/sunbur...
7266,Trees and shrubs (including roses),albizia,leaves,"curled, cupped, folded, rolled, twisted",http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/INVERT/...
24130,"Vegetables, melons, and herbs",watermelon,seedlings,"wilted, withered",http://ipm.ucanr.edu/PMG/GARDEN/VEGES/DISEASES...
20917,Trees and shrubs (including roses),wild lilac,leaves,"powdery, cottony growth",http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/INVERT/...
12746,Trees and shrubs (including roses),gold flower,flowers/blossoms,"spots, lesions, stippled",http://ipm.ucanr.edu/PMG/PESTNOTES/pn7429.html
14727,Trees and shrubs (including roses),laurel,leaves,"webbing, ""nests""",http://ipm.ucanr.edu/PMG/PESTNOTES/pn7405.html
28,Flowers (non-woody),alstroemeria,flowers/blossoms,"holes, ragged",http://ipm.ucanr.edu/PMG/PESTNOTES/pn7427.html


In [18]:
print(f'Plant type:\n{df["plant_type"].unique()}\n')
print(f'Plant part:\n{df["plant_part"].unique()}\n')
print(f'Shape:\n{df.shape}\n')
print(f'Shape of duplicated parts:\n{df[df["plant_part"].isin(["flowers/blossoms", "twigs/branches"])].shape}')

Plant type:
['Flowers (non-woody)' 'Flowers'
 'Fruit trees, nuts, berries, and grapevines'
 'Trees and shrubs (including roses)' 'Vegetables, melons, and herbs']

Plant part:
['seedlings' 'leaves' 'stems' 'roots' 'flowers/blossoms' 'overall'
 'shoots' 'buds' 'crown' 'bulbs' 'fruit' 'twigs/branches' 'trunk' 'canes'
 'nuts' 'kernels' 'needles' 'ferns' 'spears' 'tubers' 'pods']

Shape:
(24133, 5)

Shape of duplicated parts:
(4689, 5)


In [19]:
pt_map = {
    'Flowers (non-woody)': 'flowers', 
    'Flowers': 'flowers',
    'Fruit trees, nuts, berries, and grapevines': 'fruits',
    'Trees and shrubs (including roses)': 'trees',
    'Vegetables, melons, and herbs': 'vegetables'
}
df['plant_type'].replace(pt_map, inplace = True)

df['plant_damage'] = df['plant_damage'].str.replace('"nests"', 'nests')

df['plant_part'] = df['plant_part'].str.split('/')
df = df.explode('plant_part')

print(f'Shape:\n{df.shape}\n')
print(f'Plant type:\n{df["plant_type"].unique()}\n')
print(f'Plant part:\n{df["plant_part"].unique()}\n')

Shape:
(28822, 5)

Plant type:
['flowers' 'fruits' 'trees' 'vegetables']

Plant part:
['seedlings' 'leaves' 'stems' 'roots' 'flowers' 'blossoms' 'overall'
 'shoots' 'buds' 'crown' 'bulbs' 'fruit' 'twigs' 'branches' 'trunk'
 'canes' 'nuts' 'kernels' 'needles' 'ferns' 'spears' 'tubers' 'pods']



In [20]:
df.sample(10)

,plant_type,plant_name,plant_part,plant_damage,link
20514,trees,tulip tree,trunk,girdled,http://ipm.ucanr.edu/PMG/GARDEN/ENVIRON/sunbur...
4299,fruits,apricots,buds,"feeding, tunnels, boring",http://ipm.ucanr.edu/PMG/GARDEN/FRUIT/PESTS/gr...
15491,trees,manzanita,twigs,"cankers, wounds",http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASE...
1999,flowers,lily,leaves,"shrivel, droop, withered, wilted",http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/DISEASE...
7013,fruits,walnut,leaves,"galls, swell, rough",http://ipm.ucanr.edu/PMG/PESTNOTES/pn7408.html
1141,flowers,dahlia,leaves,"drop, cling to branches",http://ipm.ucanr.edu/PMG/GARDEN/FLOWERS/ENVIRO...
23930,vegetables,tomatoes,leaves,"specks, flecks, dots, stippled",http://ipm.ucanr.edu/PMG/PESTNOTES/pn7429.html
5135,fruits,citrus,buds,"feeding, tunnels, boring",http://ipm.ucanr.edu/PMG/GARDEN/FRUIT/PESTS/ci...
3819,flowers,zinnia,leaves,"spots, blotches, lesions, mottled",http://ipm.ucanr.edu/PMG/GARDEN/FLOWERS/DISEAS...
9388,trees,california buckeye,branches,"drop, weak, break, dieback, decay",http://ipm.ucanr.edu/PMG/PESTNOTES/pn74109.html
